# Kaggle Kernels: nsl-kdd ensemble-alik604

入侵检测数据集：
Dataset from https://github.com/defcom17/NSL_KDD/


more info
Sample code used: https://www.kaggle.com/meesterwaffles/nicholas-brougher-neb5211-project4

## 加载数据

In [2]:
%config IPCompleter.greedy = True # 對於接下來執行過的內容都會被列入自動提示中。
import pandas as pd
import seaborn as sns
import numpy as np
import re
import sklearn

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import matplotlib as matplot
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from sklearn.model_selection import train_test_split

In [11]:
train = pd.read_csv('https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTrain%2B.csv')
test = pd.read_csv('https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTest%2B.csv')
train.shape
test.shape 

train.columns = range(train.shape[1])
test.columns = range(test.shape[1])
labels = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
            'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
            'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
            'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
            'num_access_files', 'num_outbound_cmds', 'is_host_login',
            'is_guest_login', 'count', 'srv_count', 'serror_rate',
            'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
            'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
            'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
            'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
            'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
            'dst_host_srv_rerror_rate', 'attack_type', 'difficulty_level']# subclass - > attack_type
combined_data = pd.concat([train, test])
combined_data.shape
combined_data.head(5)

(125972, 43)

(22542, 43)

(148514, 43)

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.0,0.00,normal,15
1,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.0,0.00,neptune,19
2,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.0,0.01,normal,21
3,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,normal,21
4,0,tcp,private,REJ,0,0,0,0,0,0,...,0.07,0.07,0.00,0.00,0.00,0.00,1.0,1.00,neptune,21


去掉难度这一列，减少训练集的大小。

Reduce train size for faster trainin, remove when in production

In [15]:
combined_data.columns = labels
combined_data = combined_data.drop('difficulty_level', 1)


pandas.core.frame.DataFrame

In [16]:
combined_data.head(3)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack_type
0,0,udp,other,SF,146,0,0,0,0,0,...,1,0.0,0.60,0.88,0.00,0.00,0.00,0.0,0.00,normal
1,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.1,0.05,0.00,0.00,1.00,1.00,0.0,0.00,neptune
2,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.0,0.00,0.03,0.04,0.03,0.01,0.0,0.01,normal


In [17]:
combined_data.shape

(148514, 42)

## 数据预处理

### 完成对字符型数据的标签编码。

In [18]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

print(set(list(combined_data['attack_type']))) # use print to make it print on single line 
combined_data['attack_type'] = le.fit_transform(combined_data['attack_type'])
combined_data['protocol_type'] = le.fit_transform(combined_data['protocol_type'])
combined_data['service'] = le.fit_transform(combined_data['service'])
combined_data['flag'] = le.fit_transform(combined_data['flag'])

print('\nDescribing attack_type: ')
combined_data['attack_type'].describe()

{'guess_passwd', 'ipsweep', 'neptune', 'teardrop', 'smurf', 'xlock', 'warezclient', 'satan', 'named', 'worm', 'pod', 'rootkit', 'processtable', 'land', 'portsweep', 'snmpguess', 'spy', 'mscan', 'mailbomb', 'snmpgetattack', 'sendmail', 'loadmodule', 'back', 'multihop', 'httptunnel', 'xsnoop', 'udpstorm', 'buffer_overflow', 'nmap', 'apache2', 'xterm', 'ftp_write', 'normal', 'phf', 'ps', 'saint', 'perl', 'imap', 'warezmaster', 'sqlattack'}

Describing attack_type: 


count    148514.000000
mean         15.782243
std           4.515769
min           0.000000
25%          14.000000
50%          16.000000
75%          16.000000
max          39.000000
Name: attack_type, dtype: float64

### 特征选取

- 找与“攻击类型”相关系数最小的10个属性
- 找标准差最小的5个属性（最集中）

这些属性一般不具有预测意义，所以找出以后去掉。

In [19]:
# select least correlated

# 计算各属性的相关系数矩阵，并取绝对值，之后按攻击类型进行排序
corr_matrix = combined_data.corr().abs().sort_values('attack_type')
# tmp.head(10) # to view CORR matrix 

# 找出与攻击类型这一属性相关系数最小的10个属性
leastCorrelated = corr_matrix['attack_type'].nsmallest(10)
leastCorrelated = list(leastCorrelated.index)
leastCorrelated


# select least correlated
# 计算各属性数据的标准差，并找出最小的5行，即最不稀疏的5个属性数据
leastSTD =  combined_data.std().to_frame().nsmallest(5, columns=0)
leastSTD = list(leastSTD.transpose().columns)  #fuckin pandas.core.indexes.base.Index   -_
leastSTD

#tmp = tmp.append('num_outbound_cmds')  # might not work...
# 将特征确定为下列特征的并集
featureElimination = set(leastCorrelated + leastSTD)
len(featureElimination)
featureElimination

['num_compromised',
 'num_file_creations',
 'num_root',
 'su_attempted',
 'logged_in',
 'num_access_files',
 'dst_bytes',
 'root_shell',
 'num_shells',
 'srv_rerror_rate']

['num_outbound_cmds', 'is_host_login', 'land', 'urgent', 'num_shells']

14

{'dst_bytes',
 'is_host_login',
 'land',
 'logged_in',
 'num_access_files',
 'num_compromised',
 'num_file_creations',
 'num_outbound_cmds',
 'num_root',
 'num_shells',
 'root_shell',
 'srv_rerror_rate',
 'su_attempted',
 'urgent'}

### 确定训练集和测试集

去掉找出的属性,并将数据集分为X和y。

In [ ]:
combined_data=combined_data.drop(featureElimination,axis=1)
data_x = combined_data.drop('attack_type', axis=1)
data_y = combined_data.loc[:,['attack_type']]
#del combined_data # free mem
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=.5, random_state=42) # TODO



In [30]:
X_train.shape
X_test.shape

(74257, 27)

(74257, 27)

## 使用集成学习

弱学习器包括：
- VotingClassifier
- AdaBoostClassifier
- RandomForestClassifier
- BaggingClassifier
- ExtraTreesClassifier
- GradientBoostingClassifier

In [31]:
from sklearn import linear_model

from sklearn.ensemble import VotingClassifier

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import IsolationForest

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

import gc # 垃圾回收
gc.collect()

15

In [32]:
x = X_train
#Return the flattened underlying data as an ndarray 即将多维数组降位一维，返回视图,而不是拷贝（flatten方法返回拷贝，功能与ravel相同）
y = y_train['attack_type'].ravel() 

clf1 = DecisionTreeClassifier() 
clf2 = RandomForestClassifier(n_estimators=25, random_state=1)
clf3 = GradientBoostingClassifier()
ET = ExtraTreesClassifier(n_estimators=10, criterion='gini', max_features='auto', bootstrap=False) # .76 # without this lil fucker, Acc: 0.75 [Ensemble], 0.78 with 

eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3),('et',ET)], voting='hard') 

for clf, label in zip([clf1, clf2, clf3,ET, eclf], ['DecisionTreeClassifier', 'Random Forest', 'GradientBoostingClassifier','ExtraTreesClassifier', 'Ensemble']): 
    tmp = clf.fit(x,y)
    pred = clf.score(X_test,y_test)
    print("Acc: %0.2f [%s]" % (pred,label))

Acc: 0.99 [DecisionTreeClassifier]
Acc: 0.99 [Random Forest]
Acc: 0.99 [GradientBoostingClassifier]
Acc: 0.99 [ExtraTreesClassifier]
Acc: 0.99 [Ensemble]


In [34]:
y

array([16, 16, 16, ..., 25, 16, 16])

In [ ]:
x = X_train
y = y_train['attack_type'].ravel()

clf1 = DecisionTreeClassifier() 
clf2 = RandomForestClassifier(n_estimators=25, random_state=1)
clf3 = GradientBoostingClassifier()
ET = ExtraTreesClassifier(n_estimators=10, criterion='gini', max_features='auto', bootstrap=False) # .76 # without this lil fucker, Acc: 0.75 [Ensemble], 0.78 with 

eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3),('et',ET)], voting='hard') 

for clf, label in zip([clf1, clf2, clf3,ET, eclf], ['DecisionTreeClassifier', 'Random Forest', 'GradientBoostingClassifier','ExtraTreesClassifier', 'Ensemble']): 
    tmp = clf.fit(x,y)
    pred = clf.score(X_test,y_test)
    print("Acc: %0.2f [%s]" % (pred,label))

Acc: 0.99 [DecisionTreeClassifier]
Acc: 0.99 [Random Forest]


In [ ]:
LR = linear_model.LinearRegression()
LR.fit(X_train, y_train)
lr_score = LR.score(X_test, y_test)
print('Linear regression processing')
print('Linear regression Score: %.2f %%' % lr_score)

In [ ]:
AB = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100, learning_rate=1.0)
RF = RandomForestClassifier(n_estimators=10, criterion='entropy', max_features='auto', bootstrap=True)
ET = ExtraTreesClassifier(n_estimators=10, criterion='gini', max_features='auto', bootstrap=False)
GB = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=200, max_features='auto')

In [ ]:
AB.fit(X_train, y_train)
AB_feature = AB.feature_importances_
#AB_feature
ab_score = AB.score(X_test, y_test)

print('AdaBoostClassifier processing ,,,')
print('AdaBoostClassifier Score: %.3f %%' % ab_score)

In [ ]:
RF.fit(X_train, y_train)
RF_feature = RF.feature_importances_
#RF_feature

rf_score = RF.score(X_test, y_test)

print('RandomForestClassifier processing ,,,')
print('RandomForestClassifier Score: %.3f %%' % rf_score)

In [ ]:
ET.fit(X_train, y_train)
ET_feature = ET.feature_importances_
#ET_feature

et_score = ET.score(X_test, y_test)

print('ExtraTreesClassifier processing ,,,')
print('ExtraTreeClassifier: %.3f %%' % et_score)

In [ ]:
GB.fit(X_train, y_train)

GB_feature = GB.feature_importances_
#GB_feature

gb_score = GB.score(X_test, y_test)

print('GradientBoostingClassifier processing ,,,')
print('GradientBoostingClassifier Score: %.3f %%' % gb_score)

In [ ]:
feature_df = pd.DataFrame({'features': X_train.columns.values, # names
                           'AdaBoost' : AB_feature,
                           'RandomForest' : RF_feature,
                           'ExtraTree' : ET_feature,
                           'GradientBoost' : GB_feature
                          })
#feature_df.features
feature_df.head(5)

In [ ]:
n = 10
a_f = feature_df.nlargest(n, 'AdaBoost')
e_f = feature_df.nlargest(n, 'ExtraTree')
g_f = feature_df.nlargest(n, 'GradientBoost')
r_f = feature_df.nlargest(n, 'RandomForest')

result = pd.concat([a_f, e_f, g_f, r_f]).drop_duplicates() 
result.index
result

In [ ]:
# X_train_SF = X_train[result.index]
# X_test_SF = X_test[result.index]



selected_features = result['features'].values.tolist()
X_train_SF = X_train[selected_features]
X_test_SF = X_test[selected_features]


x = X_train_SF#.reshape(-1, 26)  # 31
y = y_train['attack_type'].ravel()


# x=x[:20000]
# y=y[:20000]


x.shape
y.size

In [ ]:
clf1 = DecisionTreeClassifier() 
clf2 = RandomForestClassifier(n_estimators=25, random_state=1)# .77
clf3 = GradientBoostingClassifier() # .76
ET = ExtraTreesClassifier(n_estimators=10, criterion='gini', max_features='auto', bootstrap=False) # .76 # without this lil fucker, Acc: 0.75 [Ensemble], 0.78 with 

eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3),('et',ET)], voting='hard') 
# n =7 with better selection; .79
# n =7 ; .77
# n =10 ; .78
# n =14 ; .77

for clf, label in zip([clf1, clf2, clf3,ET, eclf], ['DecisionTreeClassifier', 'Random Forest', 'GradientBoostingClassifier','ExtraTreesClassifier', 'Ensemble']): 
    # scores = cross_val_score(clf, x, y, cv=2, scoring='accuracy') # cv= 5 
    # print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

    tmp = clf.fit(x,y)
    pred = clf.score(X_test_SF,y_test)
    print("Acc: %0.2f [%s]" % (pred,label))